In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import quote  # 이 부분 추가
import pandas as pd
import time
import logging
from datetime import datetime
import os
import threading
import re  # 코드 상단에 추가

# PDF 저장 디렉토리와 로그 디렉토리 설정
PDF_DIR = "data/samsung"
LOG_DIR = "data/logs"

# 디렉토리 생성
os.makedirs(PDF_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

# 로그 설정
log_file = os.path.join(LOG_DIR, f"news_download_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file, encoding='utf-8'),
        # logging.StreamHandler()
    ]
)

In [2]:
# WebDriver 인스턴스를 관리하는 싱글톤 클래스
class ChromeDriverWrapper:
    """Chrome WebDriver를 관리하는 싱글톤 클래스"""
    _instance = None
    _driver = None
    _lock = threading.Lock()
    
    @classmethod
    def get_instance(cls):
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = cls()
        return cls._instance
    
    def get_driver(self):
        if self._driver is None:
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--no-sandbox")
            chrome_options.add_argument("--disable-dev-shm-usage")
            
            self._driver = webdriver.Chrome(
                service=Service(ChromeDriverManager().install()),
                options=chrome_options
            )
        return self._driver
    
    def quit_driver(self):
        if self._driver:
            try:
                self._driver.quit()
            except Exception as e:
                logging.error(f"드라이버 종료 중 오류: {str(e)}")
            finally:
                self._driver = None

In [3]:
# ... 기존 import 및 환경설정 코드 생략 ...

class SamsungSemiconNewsScraper:
    def __init__(self):
        logging.info("스크래퍼 초기화 시작")
        self.categories = [
            {
                "name": "프레스센터",
                "url": "https://news.samsungsemiconductor.com/kr/category/%eb%89%b4%ec%8a%a4/"
            },
            {
                "name": "문화",
                "url": "https://news.samsungsemiconductor.com/kr/category/%eb%ac%b8%ed%99%94/"
            },
            {
                "name": "ESG",
                "url": "https://news.samsungsemiconductor.com/kr/category/esg/"
            }
        ]
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--window-size=1920,1080")
        chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
        self.driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
            options=chrome_options
        )
        self.wait = WebDriverWait(self.driver, 20)
        self.articles = []
        logging.info("스크래퍼 초기화 완료")

    def get_article_content(self, url):
        logging.info(f"기사 내용 추출 시작: {url}")
        try:
            self.driver.get(url)
            time.sleep(2)
            content_container = self.wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.content_view > div.content_desc"))
            )
            paragraphs = content_container.find_elements(By.TAG_NAME, "p")
            content_texts = [p.text.strip() for p in paragraphs if p.text.strip()]
            content = '\n'.join(content_texts)
            return content if content else "내용 추출 실패"
        except Exception as e:
            logging.error(f"본문 추출 중 오류: {str(e)}")
            return "내용 추출 실패"

    def collect_articles_from_category(self, category_name, category_url, page_limit=None):
        total_collected_pages = 0
        current_page = 1

        # page_limit이 설정된 경우 목표 페이지 수를 로깅
        if page_limit:
            logging.info(f"[{category_name}] 목표 페이지 수: {page_limit}")

        while True:
            # 현재 페이지 URL 구성
            if current_page == 1:
                page_url = category_url
            else:
                page_url = f"{category_url}page/{current_page}/"
            
            logging.info(f"[{category_name}] {current_page}페이지 접속: {page_url}")
            self.driver.get(page_url)
            time.sleep(2)

            # 404 페이지 체크
            try:
                error_page = self.driver.find_element(By.CSS_SELECTOR, "div.page_404")
                if error_page:
                    logging.info(f"[{category_name}] {current_page}페이지에서 404 에러 발생. 수집을 종료합니다.")
                    break
            except:
                pass  # 404 페이지가 아닌 경우 정상 진행

            # 기사 수집
            try:
                articles = self.driver.find_elements(By.CSS_SELECTOR, "ul.article_list > li.article_item")
                if not articles:
                    logging.info(f"[{category_name}] {current_page}페이지에 기사가 없습니다.")
                    break
                
                logging.info(f"[{category_name}] {current_page}페이지 기사 수: {len(articles)}")
                article_infos = []
                for article in articles:
                    try:
                        link_element = article.find_element(By.TAG_NAME, "a")
                        article_url = link_element.get_attribute("href")
                        title = article.find_element(By.CSS_SELECTOR, "p.title").text.strip()
                        date = article.find_element(By.CSS_SELECTOR, "span.date").text.strip()
                        try:
                            category = article.find_element(By.CSS_SELECTOR, "span.category").text.strip()
                        except:
                            category = ""
                        try:
                            desc = article.find_element(By.CSS_SELECTOR, "p.desc").text.strip()
                        except:
                            desc = ""
                        article_infos.append({
                            "category_group": category_name,
                            "title": title,
                            "url": article_url,
                            "date": date,
                            "category": category,
                            "desc": desc
                        })
                    except Exception as e:
                        logging.error(f"기사 리스트 정보 추출 중 오류: {str(e)}")

                for info in article_infos:
                    content = self.get_article_content(info["url"])
                    info["content"] = content
                    self.articles.append(info)
                    logging.info(f"기사 추가: {info['title']}")

                total_collected_pages += 1

                # 페이지 제한 체크
                if page_limit:
                    if current_page >= page_limit:
                        logging.info(f"[{category_name}] 목표 페이지 수({page_limit})에 도달하여 수집을 종료합니다.")
                        break
                    else:
                        logging.info(f"[{category_name}] {current_page}/{page_limit} 페이지 수집 완료")
                else:
                    logging.info(f"[{category_name}] {current_page}페이지 수집 완료")

                current_page += 1

            except Exception as e:
                logging.error(f"페이지 {current_page} 처리 중 오류: {str(e)}")
                break

        logging.info(f"[{category_name}] 기사 수집 완료 (총 {total_collected_pages}페이지)")

    def search_news(self, page_limit=None):
        for cat in self.categories:
            logging.info(f"{cat['name']} 기사 수집 시작")
            self.collect_articles_from_category(cat["name"], cat["url"], page_limit=page_limit)
            logging.info(f"{cat['name']} 기사 수집 완료")

    def save_to_csv(self, filename):
        logging.info(f"CSV 파일 저장 시작: {filename}")
        try:
            if not self.articles:
                logging.warning("저장할 기사가 없습니다")
                return
            df = pd.DataFrame(self.articles)
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            logging.info("CSV 파일 저장 완료")
        except Exception as e:
            logging.error(f"CSV 파일 저장 중 오류: {str(e)}")

    def __del__(self):
        try:
            self.driver.quit()
        except:
            pass

In [4]:
try:
    scraper = SamsungSemiconNewsScraper()
    scraper.search_news(page_limit=85)  # 또는 scraper.search_news()
    scraper.save_to_csv("data/samsung/leadership_news.csv")
finally:
    scraper.__del__()